In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.decomposition import PCA
DATA_DIR = os.path.join(os.getcwd(), 'data')

## Import Labeled Data

In [3]:
# data = pd.read_csv(os.path.join(DATA_DIR, 'dummy_labeled_ind1-3.csv'), index_col=0)
data = pd.read_csv(os.path.join(DATA_DIR, 'ind4_clustering_10PCs.csv'))
data

,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,PC_10,cluster_label
0,-18.265232,0.179890,2.512753,-0.017214,-1.253427,5.103237,3.069733,-1.498417,-0.213276,-1.921352,B
1,-11.648572,-2.388439,0.988511,3.860645,1.748230,0.217374,-0.771974,0.272593,1.387455,-0.643988,B
2,2.597364,-1.249371,-3.363064,-2.655913,-1.898509,-0.066221,-3.849705,2.200412,-2.473733,-0.130113,L1
3,-3.417240,-4.438395,0.636533,3.351176,0.887752,-7.494596,-0.488962,0.585061,2.802112,-1.061658,B
4,7.382528,-0.940683,-9.480106,-6.126398,2.831281,1.671729,0.735573,-5.483888,3.156084,-4.968595,L1
...,...,...,...,...,...,...,...,...,...,...,...
4009,8.322987,-3.214379,-0.689009,-0.262546,1.024876,-2.575015,-1.277584,2.146089,1.676037,0.391478,L1
4010,-11.008588,-1.900832,-2.101015,2.335349,1.157123,-4.229745,0.003551,1.722289,-0.468227,2.221812,B
4011,9.320722,-5.689082,2.899196,2.080614,-1.207569,-1.014811,5.033344,2.736327,-3.361486,1.148018,L1
4012,7.061739,10.454308,7.367766,7.623241,1.983226,1.349209,-3.744541,-8.520135,2.017838,1.947278,L2


In [4]:
data['cluster_label'].value_counts()

cluster_label
L1              1931
B               1429
L2               572
Unclassified      82
Name: count, dtype: int64

In [5]:
data.describe()

,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,PC_10
count,4014.000000,4014.000000,4014.000000,4014.000000,4014.000000,4014.000000,4014.000000,4014.000000,4014.000000,4014.000000
mean,0.006364,-0.021141,-0.078401,0.083243,0.065213,0.008302,0.070433,-0.011541,-0.050991,-0.019961
std,8.610749,6.167741,4.481838,4.296681,3.434765,3.133784,2.905260,2.792151,2.708235,2.444898
min,-29.659653,-9.499155,-13.907784,-43.566033,-57.245970,-14.597362,-21.256365,-15.068132,-8.321540,-8.657476
25%,-7.966932,-3.430494,-2.636772,-1.408544,-0.966391,-1.665316,-1.810213,-1.357253,-1.739949,-1.497361
50%,3.099684,-1.893019,-0.082368,0.591807,0.255896,0.038975,0.065079,0.198538,-0.242970,-0.049414
75%,7.005078,-0.071587,2.207989,2.314132,1.397715,1.754820,2.037075,1.666112,1.390688,1.504905
max,17.917305,22.768726,34.417159,10.826321,10.860815,11.885671,8.529058,11.644395,20.034509,9.515228


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4014 entries, 0 to 4013
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PC_1           4014 non-null   float64
 1   PC_2           4014 non-null   float64
 2   PC_3           4014 non-null   float64
 3   PC_4           4014 non-null   float64
 4   PC_5           4014 non-null   float64
 5   PC_6           4014 non-null   float64
 6   PC_7           4014 non-null   float64
 7   PC_8           4014 non-null   float64
 8   PC_9           4014 non-null   float64
 9   PC_10          4014 non-null   float64
 10  cluster_label  4014 non-null   object 
dtypes: float64(10), object(1)
memory usage: 345.1+ KB


## PreProcessing

Split X, y and turn labels from string to int with values in range (0, n_classes-1).

In [7]:
from preprocessing import split_X_y

X, y = split_X_y(data, label_name='cluster_label')

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import LabelBinarizer, StandardScaler
import numpy as np
# lb = LabelBinarizer()
# y = lb.fit_transform(data['cluster_label'])

# scaler = StandardScaler()
# data_rescaled = scaler.fit_transform(X)

# remove if pca is already done
# pca = PCA(n_components=10)
# pca.fit(data_rescaled)
# X_pca = pca.transform(data_rescaled)
# print(X_pca.shape)

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=148, test_size=0.33, stratify=y, shuffle=True)
print(x_test.shape)

(1325, 10)


In [10]:
y_train

array([0, 1, 1, ..., 3, 3, 3])

In [11]:
estimator1 = OneVsRestClassifier(LinearSVC(C=0.1, random_state=148, tol=1e-5, multi_class='ovr'))
estimator1.fit(x_train, y_train)


from sklearn.metrics import classification_report, make_scorer, matthews_corrcoef
mcc_scorer = make_scorer(matthews_corrcoef)

y_pred1 = estimator1.predict(x_test)

print(classification_report(y_test, y_pred1))
score = matthews_corrcoef(y_test, y_pred1)
print(score)

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       472
           1       0.98      0.99      0.99       189
           2       0.93      1.00      0.96        27
           3       1.00      0.98      0.99       637

    accuracy                           0.99      1325
   macro avg       0.97      0.99      0.98      1325
weighted avg       0.99      0.99      0.99      1325

0.9794769142647508


In [12]:
# estimator2 = LinearSVC(C=0.1, random_state=148, tol=1e-5, multi_class='ovr')
# estimator2.fit(x_train, y_train)
#
# y_pred2 = estimator2.predict(x_test)
#
# print(classification_report(y_test, y_pred2))
# print(matthews_corrcoef(y_test, y_pred2))

In [13]:
# estimator = OneVsRestClassifier(SVC(kernel='rbf', C=0.1, gamma=0.01, random_state=148))
estimator3 = OneVsRestClassifier(LinearSVC(C=0.1, random_state=148, tol=1e-5, multi_class='ovr', max_iter=100000))
cv3 = StratifiedKFold(n_splits=5, shuffle=True, random_state=148)
    # -- Evaluate the score by cross-validation
score3 = cross_val_score(estimator3, X, y, scoring='balanced_accuracy', cv=cv3, n_jobs=-1)
mean_score3 = score3.mean()

print(mean_score3)

0.9808486205211228


In [14]:
print(score3.std(ddof=1) / np.sqrt(len(score3)))

0.005210917409861901


In [15]:
# estimator4 = LinearSVC(C=0.1, random_state=148, tol=1e-5, multi_class='ovr', max_iter=100000)
# cv4 = StratifiedKFold(n_splits=5, shuffle=True, random_state=148)
#     # -- Evaluate the score by cross-validation
# score4 = cross_val_score(estimator4, X, y, scoring='f1_weighted', cv=cv4, n_jobs=-1)
# mean_score4 = score4.mean()
#
# print(mean_score4)

In [16]:
# print(score4.std(ddof=1) / np.sqrt(len(score4)))

In [17]:
from sklearn.naive_bayes import GaussianNB

estimator5 = GaussianNB()
cv5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=148)
    # -- Evaluate the score by cross-validation
score5 = cross_val_score(estimator5, X, y, scoring='balanced_accuracy', cv=cv5, n_jobs=-1)
mean_score5 = score5.mean()

print(mean_score5)
print(score5.std(ddof=1) / np.sqrt(len(score5)))

0.9653607894846032
0.0033681524468305627


In [18]:
from sklearn.linear_model import LogisticRegression

estimator6 = OneVsRestClassifier(LogisticRegression(solver='saga', C=0.1, random_state=148, penalty='l2', multi_class='ovr',
                               max_iter=1000, n_jobs=-1), n_jobs=-1)
cv6 = StratifiedKFold(n_splits=5, shuffle=True, random_state=148)
    # -- Evaluate the score by cross-validation
score6 = cross_val_score(estimator6, X, y, scoring='balanced_accuracy', cv=cv6, n_jobs=-1)
mean_score6 = score6.mean()

print(mean_score6)
print(score6.std(ddof=1) / np.sqrt(len(score6)))

/home/daphne/.pyenv/versions/3.9.1/envs/machineLearningProjects/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/daphne/.pyenv/versions/3.9.1/envs/machineLearningProjects/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/daphne/.pyenv/versions/3.9.1/envs/machineLearningProjects/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/daphne/.pyenv/versions/3.9.1/envs/machineLearningProjects/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/daphne/.pyenv/versions/3.9.1/envs/machineLearningProjects/lib/python3.9/site-packages/

0.9795401241001551
0.005745266550964559


## Optimize models